# <center>PROJECT 2 - IMAGE PROCESSING</center>

&nbsp;
### I. Sinh viên thực hiện
- Họ và tên: Võ Quốc Bình
- MSSV: 21127233
- Lớp học phần: 21CLC08 - Toán ứng dụng và thống kê cho CNTT

### II. Thư viện sử dụng

In [59]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

### III. Xử lí đầu vào

#### 1. Kiểm tra tên file có trong thư mục không

In [60]:
def is_valid_file(file_path):
    try:
        # Mở file để kiểm tra xem có thể đọc được hay không
        with open(file_path, 'r'):
            return True
    except FileNotFoundError:
        # File không tồn tại
        return False
    except IsADirectoryError:
        # Đường dẫn trỏ đến thư mục, không phải là tệp
        return False
    except:
        return False


#### 2. Nhập tên file

In [61]:
def input_data():
    # Nhập tên file
    while True:
        filename = input('Enter name of an image: ')
        typefile = filename[len(filename)-3:]
        if ((typefile == 'jpg' or typefile == 'png') and (is_valid_file(filename))):
            break

    # Nhập lựa chọn từ 0 đến 7. Nếu khác giá trị từ 0 đén 7, gán lựa chọn = 0
    print("Options: ")
    print("0. All options")
    print("1. Change the brightness of image")
    print("2. Change the contrast of image")
    print("3. Flip image (vertical/horizontal)")
    print("4. Convert RGB image to grayscale/sepia")
    print("5. Blur/Sharpen image")
    print("6. Crop image to size (crop in center)")
    print("7. Crop the photo to circular frame")
    print("8. Crop the photo to ellipse frame")

    option = int(input("Input your option: "))
    if not (option <= 8 and option >= 0):
        option = 0

    return filename, option


#### 3. Tiền xử lí

In [62]:
def change_to_array(filename):
    # Mở hình ảnh
    image = Image.open(filename)
    # Convert to numpy array (3D matrix)
    image_array = np.array(image)
    return image_array

### IV. Xuất ảnh

In [63]:
def output_image(image_array):
    image = Image.fromarray(image_array.astype(np.uint8))
    return image

### V. Lưu ảnh

In [64]:
def save_image(image_array, filename, name_process):
    image = output_image(image_array)
    output_type = filename[len(filename)-3:]
    name = filename[:len(filename)-4]

    image_save = name + "_"+name_process + "." + output_type
    image.save(image_save)


### VI. Xử lí các chức năng ảnh

#### 1. Thay đổi độ sáng cho ảnh

In [65]:
def change_brightness(image_array, brightness=25):
    output_image = image_array.copy()

    output_image = np.clip(output_image+float(brightness), 0, 255)
    return output_image


#### 2. Thay đổi độ tương phản 

In [66]:
def change_constract(image_array, contrast=25):
    output_image = image_array.copy()

    factor = (259 * (255 + contrast)) / (255 * (259 - contrast))
    # Điều chỉnh độ tương phản của ảnh
    output_image = float(factor)*output_image - (factor-1) * 128

    # Giới hạn giá trị pixel trong khoảng từ 0 đến 255
    output_image = np.clip(output_image, 0, 255)

    return output_image


#### 3. Lật ảnh

##### a. Lật dọc

In [67]:
def flip_vertically(image_array):
    output_image = image_array[::-1, :]
    return output_image


##### b. Lật ngang

In [68]:
def flip_horizontally(image_array):
    output_image = image_array[:, ::-1]
    return output_image


#### 4. Chuyển ảnh màu RGB

##### a. Màu xám

In [69]:
def change_to_grayscale(image_array):
    gray_image = np.zeros(image_array.shape)
    R = np.array(image_array[:, :, 0])
    G = np.array(image_array[:, :, 1])
    B = np.array(image_array[:, :, 2])

    R = (R * .299)
    G = (G * .587)
    B = (B * .114)

    Avg = (R+G+B)
    gray_image = image_array.copy()

    for i in range(3):
        gray_image[:, :, i] = Avg

    return gray_image


##### b. Màu sepia

In [70]:
def change_to_sepia(image_array):
    # Create a new array for the sepia tone image
    sepia_array = np.empty_like(image_array)
    sepia_matrix = np.array([[0.393, 0.769, 0.189],
                            [0.349, 0.686, 0.168],
                            [0.272, 0.534, 0.131]])

    # Apply the sepia transformation to each pixel
    sepia_array = image_array[..., :3] @ sepia_matrix.T

    # Clip the pixel values to the valid range (0 to 255)
    sepia_array = np.clip(sepia_array, 0, 255)
    
    return sepia_array


#### 5. Làm mờ/ Làm nét ảnh

##### a. Làm mờ ảnh

In [71]:
def blur_image(image_array, iteration=1):
    blur_kernel = np.array([[[1], [2], [1]],
                            [[2], [4], [2]],
                            [[1], [2], [1]]])
    blur_kernel = blur_kernel/(np.sum(blur_kernel))

    output_image = image_array.copy()

    for _ in range(iteration):
        temp = output_image.copy()
        output_image = temp.copy()
        cols = image_array.shape[0]
        rows = image_array.shape[1]

        padded_image = np.zeros(
            (output_image.shape[0] + 2, output_image.shape[1] + 2, output_image.shape[2]))
        padded_image[1:-1, 1:-1, :] = output_image

        for col in range(0, cols):
            for row in range(0, rows):
                neighbor = padded_image[col:col+3, row:row+3]
                output_image[col][row] = (
                    neighbor * blur_kernel).sum(axis=1).sum(axis=0)

    return output_image


##### b. Làm sắc nét ảnh

In [72]:
def sharpen_image(image_array, iteration=1):
    # Define the sharpening kernel (inverse of the blur kernel)
    sharpen_kernel = np.array([[[0], [-1], [0]],
                            [[-1], [5], [-1]],
                            [[0], [-1], [0]]])

    output_image = image_array.copy()

    for _ in range(iteration):
        temp = output_image.copy()
        output_image = temp.copy()
        cols = image_array.shape[0]
        rows = image_array.shape[1]

        padded_image = np.zeros(
            (output_image.shape[0] + 2, output_image.shape[1] + 2, output_image.shape[2]))
        padded_image[1:-1, 1:-1, :] = output_image

        for col in range(0, cols):
            for row in range(0, rows):
                neighbor = padded_image[col:col + 3, row:row + 3]
                output_image[col][row] = np.clip((
                    neighbor * sharpen_kernel).sum(axis=1).sum(axis=0),0,255)

    return output_image


#### 6. Cắt hình ở trung tâm

In [73]:
def crop_center(image_array):
    edge_size = int(min(image_array.shape[0],image_array.shape[1])/2)

    height, width = image_array.shape[:2]

    start_h = (height - edge_size) // 2
    start_w = (width - edge_size) // 2

    cropped_image = image_array[start_h:start_h +
                                edge_size, start_w:start_w+edge_size]

    return cropped_image


#### 7. Cắt hình tròn

In [74]:
def crop_circle(image_array):
    # get the picture frame
    edge_size = int(min(image_array.shape[0], image_array.shape[1]))

    # get the size of original picture
    height, width = image_array.shape[:2]

    start_h = (height - edge_size) // 2
    start_w = (width - edge_size) // 2

    cropped_image = image_array[start_h:start_h +
                                edge_size, start_w:start_w+edge_size]

    # create a matrix to mark element int circle frame
    mask = np.zeros_like(cropped_image, dtype=bool)
    radius = edge_size//2
    center = (radius, radius)
    for y in range(edge_size):
        for x in range(edge_size):
            # formula circle area: (x-a)^2 + (y-b)^2 <= r^2
            if (x - center[1])**2 + (y - center[0])**2 <= radius**2:
                mask[y, x] = True

    # apply the mask to the image
    cropped_image[~mask] = 0

    return cropped_image


#### 8. Cắt 2 hình ellipse chéo nhau

In [75]:
def crop_two_cross_ellipse(image_array):
    # get the picture frame
    edge_size = int(min(image_array.shape[0], image_array.shape[1]))

    # get the size of original picture
    height, width = image_array.shape[:2]

    start_h = (height - edge_size) // 2
    start_w = (width - edge_size) // 2

    cropped_image = image_array[start_h:start_h +
                                edge_size, start_w:start_w+edge_size]

    # create a matrix to mark element int circle frame
    mask = np.zeros_like(cropped_image, dtype=bool)
    radius = edge_size/2
    center = (radius, radius)

    major_axis = radius*1.25
    minor_axis = radius*0.671875
    angle_radians_prime = np.deg2rad(45)
    angle_radians_sub = np.deg2rad(-45)

    # crop prime ellipse
    for y in range(edge_size):
        for x in range(edge_size):
            distance_1 = ((x-center[0])*np.cos(angle_radians_prime) +
                          (y-center[1])*np.sin(angle_radians_prime))**2/minor_axis**2
            distance_2 = ((x-center[0])*np.sin(angle_radians_prime) -
                          (y-center[1])*np.cos(angle_radians_prime))**2/major_axis**2

            if(distance_1+distance_2) <= 1:
                mask[y, x] = True

    # crop sub ellipse
    for y in range(edge_size):
        for x in range(edge_size):
            distance_1 = ((x-center[0])*np.sin(angle_radians_sub) +
                          (y-center[1])*np.cos(angle_radians_sub))**2/minor_axis**2
            distance_2 = ((x-center[0])*np.cos(angle_radians_sub) -
                          (y-center[1])*np.sin(angle_radians_sub))**2/major_axis**2

            if(distance_1+distance_2) <= 1:
                mask[y, x] = True

    # apply the mask to the image
    cropped_image[~mask] = 0

    return cropped_image


### VII. Xử lí các lựa chọn

In [76]:
def handle_option_0(filename):
    direction = input("In option 3 choose type flip (vertical or horizontal):")
    if direction == "vertical":
        direction = direction
    else:
        direction = "horizontal"
    handle_option_1(filename)
    handle_option_2(filename)
    handle_option_3(filename, direction)
    handle_option_4(filename)
    handle_option_5(filename)
    handle_option_6(filename)
    handle_option_7(filename)
    handle_option_8(filename)


def handle_option_1(filename):
    image_array = change_to_array(filename)
    output_image = change_brightness(image_array)
    save_image(output_image, filename, "brightness")


def handle_option_2(filename):
    image_array = change_to_array(filename)
    output_image = change_constract(image_array)
    save_image(output_image, filename, "constract")


def handle_option_3(filename, direction):
    if direction == "vertical":
        image_array = change_to_array(filename)
        output_image = flip_vertically(image_array)
        save_image(output_image, filename, "vertical")
    else:
        image_array = change_to_array(filename)
        output_image = flip_horizontally(image_array)
        save_image(output_image, filename, "horizontal")


def handle_option_4(filename):
    image_array = change_to_array(filename)
    output_image = change_to_grayscale(image_array)
    save_image(output_image, filename, "grayscale")
    
    image_array = change_to_array(filename)
    output_image = change_to_sepia(image_array)
    save_image(output_image, filename, "sepia")


def handle_option_5(filename):
    image_array = change_to_array(filename)
    output_image = blur_image(image_array)
    save_image(output_image, filename, "blur")

    image_array = change_to_array(filename)
    output_image = sharpen_image(image_array)
    save_image(output_image, filename, "sharpen")


def handle_option_6(filename):
    image_array = change_to_array(filename)
    output_image = crop_center(image_array)
    save_image(output_image, filename, "crop_center")


def handle_option_7(filename):
    image_array = change_to_array(filename)
    output_image = crop_circle(image_array)
    save_image(output_image, filename, "circle")


def handle_option_8(filename):
    image_array = change_to_array(filename)
    output_image = crop_two_cross_ellipse(image_array)
    save_image(output_image, filename, "ellipse_cross")


### VIII. Hàm main demo chương trình

In [77]:
def main():
    filename, option = input_data()
    if option == 0:
        handle_option_0(filename)
    elif option == 1:
        handle_option_1(filename)
    elif option == 2:
        handle_option_2(filename)
    elif option == 3:
        direction = input(
            "In option 3 choose type flip (vertical or horizontal):")
        if direction == "vertical":
            direction = direction
        else:
            direction = "horizontal"
        handle_option_3(filename, direction)
    elif option == 4:
        handle_option_4(filename)
    elif option == 5:
        handle_option_5(filename)
    elif option == 6:
        handle_option_6(filename)
    elif option == 7:
        handle_option_7(filename)
    elif option == 8:
        handle_option_8(filename)


main()


Options: 
0. All options
1. Change the brightness of image
2. Change the contrast of image
3. Flip image (vertical/horizontal)
4. Convert RGB image to grayscale/sepia
5. Blur/Sharpen image
6. Crop image to size (crop in center)
7. Crop the photo to circular frame
8. Crop the photo to ellipse frame
